Ricky's Working Notebook

In [48]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

In [49]:
con = sqlite3.connect('Data/im.db')

In [50]:
pd.read_sql('''

SELECT *
FROM sqlite_master

''', con)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [51]:
#Gotta check on my little brother
pd.read_sql('''

SELECT *
FROM principals
    JOIN persons
        USING(person_id)
    JOIN movie_basics
        USING(movie_id)
WHERE primary_name = 'Nick Robinson'

''', con)

,movie_id,ordering,person_id,category,job,characters,primary_name,birth_year,death_year,primary_profession,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0835802,2,nm3538718,actor,None,"[""Taylor Ogburn""]",Nick Robinson,1995.0,None,"actor,soundtrack",Krystal,Krystal,2017,90.0,"Comedy,Drama"
1,tt2179116,1,nm3538718,actor,None,"[""Joe""]",Nick Robinson,1995.0,None,"actor,soundtrack",The Kings of Summer,The Kings of Summer,2013,95.0,"Adventure,Comedy,Drama"
2,tt2866708,4,nm3538718,actor,None,"[""Philip""]",Nick Robinson,1995.0,None,"actor,soundtrack",Strange But True,Strange But True,2019,NaN,Thriller
3,tt4353270,2,nm3538718,actor,None,"[""Lance Zutterland""]",Nick Robinson,1995.0,None,"actor,soundtrack",Echo Boomers,Echo Boomers,2019,NaN,"Action,Crime,Drama"
4,tt4630444,1,nm3538718,actor,None,"[""Charlie Mills""]",Nick Robinson,1995.0,None,"actor,soundtrack",Being Charlie,Being Charlie,2015,97.0,"Drama,Romance"
5,tt5001718,2,nm3538718,actor,None,"[""Olly""]",Nick Robinson,1995.0,None,"actor,soundtrack","Everything, Everything","Everything, Everything",2017,96.0,"Drama,Romance"
6,tt5164432,1,nm3538718,actor,None,"[""Simon""]",Nick Robinson,1995.0,None,"actor,soundtrack","Love, Simon","Love, Simon",2018,110.0,"Comedy,Drama,Romance"
7,tt6566768,3,nm3538718,actor,None,"[""Jan""]",Nick Robinson,1995.0,None,"actor,soundtrack",Native Son,Native Son,2019,104.0,Drama


In [52]:
pd.read_sql('''

SELECT *
FROM movie_basics
LIMIT 15
        
''', con)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
5,tt0111414,A Thin Life,A Thin Life,2018,75.0,Comedy
6,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller"
7,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy"
8,tt0139613,O Silêncio,O Silêncio,2012,NaN,"Documentary,History"
9,tt0144449,Nema aviona za Zagreb,Nema aviona za Zagreb,2012,82.0,Biography


In [53]:
#reading in financial data csv
tn_data = pd.read_csv('Data/tn.movie_budgets.csv')
bom_data = pd.read_csv('Data/bom.movie_gross.csv')
kaggle_data = pd.read_csv('Data/movie_statistic_dataset.csv')

In [65]:

tn_data['production_budget'] = tn_data['production_budget'].str.replace('$', '').str.replace(',', '').astype(float)
tn_data['domestic_gross'] = tn_data['domestic_gross'].str.replace('$', '').str.replace(',', '').astype(float)
tn_data['worldwide_gross'] = tn_data['worldwide_gross'].str.replace('$', '').str.replace(',', '').astype(float)


In [69]:
tn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5782 non-null   int64  
 1   release_date       5782 non-null   object 
 2   movie              5782 non-null   object 
 3   production_budget  5782 non-null   float64
 4   domestic_gross     5782 non-null   float64
 5   worldwide_gross    5782 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 271.2+ KB


In [67]:
tn_data = tn_data[tn_data['production_budget'] != 0]

In [84]:
tn_and_bom = tn_data.merge(bom_data, left_on='movie', right_on='title', how='left')

In [85]:
#tn_and_bom[(tn_and_bom['domestic_gross_x']==0) & (tn_and_bom['domestic_gross_y'].notnull)]
tn_and_bom = tn_and_bom[(tn_and_bom['domestic_gross_x'] != 0) | (tn_and_bom['domestic_gross_y'].notnull())]

In [87]:
#Now we learned that the bom data set only has info for 8 movies not in the tn data set so we can stop looking. 
tn_and_bom[tn_and_bom['domestic_gross_x']==0 & (tn_and_bom['domestic_gross_y'].notnull())]

,id,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,title,studio,domestic_gross_y,foreign_gross,year
2850,51,"Nov 14, 2014",Wolves,18000000.0,0.0,94953.0,Wolves,KE,12100.0,NaN,2014.0
3153,54,"Dec 6, 2013",Khumba,15000000.0,0.0,20898221.0,Khumba,MNE,1000.0,27200000,2013.0
4463,64,"Jun 6, 2014",Anna,4357373.0,0.0,1200000.0,Anna,VE,4300.0,NaN,2014.0
4768,69,"Sep 18, 1967",Point Blank,3000000.0,0.0,0.0,Point Blank,Magn.,708000.0,8500000,2011.0
4865,66,"Jan 19, 2016",Eden,2300000.0,0.0,0.0,Eden,BG,65500.0,NaN,2015.0
5330,31,"Dec 31, 2012",Trance,950000.0,0.0,0.0,Trance,FoxS,2300000.0,21900000,2013.0
5351,52,"Dec 31, 2012",Snitch,850000.0,0.0,0.0,Snitch,LG/S,42900000.0,NaN,2013.0
5415,16,"Mar 4, 2016",They Will Have to Kill Us First,600000.0,0.0,7943.0,They Will Have to Kill Us First,BBC,7500.0,NaN,2016.0


In [88]:
tn_data = tn_data[tn_data['domestic_gross'] != 0]

In [89]:
tn_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5234 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5234 non-null   int64  
 1   release_date       5234 non-null   object 
 2   movie              5234 non-null   object 
 3   production_budget  5234 non-null   float64
 4   domestic_gross     5234 non-null   float64
 5   worldwide_gross    5234 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 286.2+ KB


In [90]:
money_data = kaggle_data.merge(tn_data, left_on='movie_title', right_on='movie', how='outer')

In [91]:
money_data.head()

,movie_title,production_date,genres,runtime_minutes,director_name,director_professions,director_birthYear,director_deathYear,movie_averageRating,movie_numerOfVotes,approval_Index,Production budget $,Domestic gross $,Worldwide gross $,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,"Jul 17, 2009",(500) Days of Summer,7500000.0,32425665.0,34439060.0
1,10 Cloverfield Lane,2016-01-04,"Drama,Horror,Mystery",103.0,Dan Trachtenberg,"music_department,director,writer",\N,alive,7.2,333495.0,6.605087,15000000.0,72082999.0,108286422.0,54.0,"Mar 11, 2016",10 Cloverfield Lane,5000000.0,72082999.0,108286422.0
2,10 Days in a Madhouse,2015-11-11,Drama,111.0,Timothy Hines,"visual_effects,director,writer",1960,alive,5.8,2797.0,3.254874,12000000.0,14616.0,14616.0,48.0,"Nov 11, 2015",10 Days in a Madhouse,12000000.0,14616.0,14616.0
3,10 Things I Hate About You,1999-03-31,"Comedy,Drama,Romance",97.0,Gil Junger,"producer,director,miscellaneous",1954,alive,7.3,349513.0,6.723852,13000000.0,38177966.0,60413950.0,63.0,"Mar 31, 1999",10 Things I Hate About You,13000000.0,38177966.0,60413950.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.0,"Mar 7, 2008","10,000 B.C.",105000000.0,94784201.0,269065678.0


In [93]:
money_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5758 entries, 0 to 5757
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_title           4527 non-null   object 
 1   production_date       4527 non-null   object 
 2   genres                4527 non-null   object 
 3   runtime_minutes       4527 non-null   float64
 4   director_name         4527 non-null   object 
 5   director_professions  4527 non-null   object 
 6   director_birthYear    4527 non-null   object 
 7   director_deathYear    4527 non-null   object 
 8   movie_averageRating   4527 non-null   float64
 9   movie_numerOfVotes    4527 non-null   float64
 10  approval_Index        4527 non-null   float64
 11  Production budget $   4527 non-null   float64
 12  Domestic gross $      4527 non-null   float64
 13  Worldwide gross $     4527 non-null   float64
 14  id                    5434 non-null   float64
 15  release_date         

In [62]:
money_data.head()

,movie_title,production_date,genres,runtime_minutes,director_name,director_professions,director_birthYear,director_deathYear,movie_averageRating,movie_numerOfVotes,approval_Index,Production budget $,Domestic gross $,Worldwide gross $,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,"Nov 20, 2015",#Horror,1500000.0,$0,$0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,"Jul 17, 2009",(500) Days of Summer,7500000.0,"$32,425,665","$34,439,060"
2,10 Cloverfield Lane,2016-01-04,"Drama,Horror,Mystery",103.0,Dan Trachtenberg,"music_department,director,writer",\N,alive,7.2,333495.0,6.605087,15000000.0,72082999.0,108286422.0,54.0,"Mar 11, 2016",10 Cloverfield Lane,5000000.0,"$72,082,999","$108,286,422"
3,10 Days in a Madhouse,2015-11-11,Drama,111.0,Timothy Hines,"visual_effects,director,writer",1960,alive,5.8,2797.0,3.254874,12000000.0,14616.0,14616.0,48.0,"Nov 11, 2015",10 Days in a Madhouse,12000000.0,"$14,616","$14,616"
4,10 Things I Hate About You,1999-03-31,"Comedy,Drama,Romance",97.0,Gil Junger,"producer,director,miscellaneous",1954,alive,7.3,349513.0,6.723852,13000000.0,38177966.0,60413950.0,63.0,"Mar 31, 1999",10 Things I Hate About You,13000000.0,"$38,177,966","$60,413,950"


In [64]:
money_data.head()

,movie_title,production_date,genres,runtime_minutes,director_name,director_professions,director_birthYear,director_deathYear,movie_averageRating,movie_numerOfVotes,approval_Index,Production budget $,Domestic gross $,Worldwide gross $,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,"Nov 20, 2015",#Horror,1500000.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,"Jul 17, 2009",(500) Days of Summer,7500000.0,32425665.0,34439060.0
2,10 Cloverfield Lane,2016-01-04,"Drama,Horror,Mystery",103.0,Dan Trachtenberg,"music_department,director,writer",\N,alive,7.2,333495.0,6.605087,15000000.0,72082999.0,108286422.0,54.0,"Mar 11, 2016",10 Cloverfield Lane,5000000.0,72082999.0,108286422.0
3,10 Days in a Madhouse,2015-11-11,Drama,111.0,Timothy Hines,"visual_effects,director,writer",1960,alive,5.8,2797.0,3.254874,12000000.0,14616.0,14616.0,48.0,"Nov 11, 2015",10 Days in a Madhouse,12000000.0,14616.0,14616.0
4,10 Things I Hate About You,1999-03-31,"Comedy,Drama,Romance",97.0,Gil Junger,"producer,director,miscellaneous",1954,alive,7.3,349513.0,6.723852,13000000.0,38177966.0,60413950.0,63.0,"Mar 31, 1999",10 Things I Hate About You,13000000.0,38177966.0,60413950.0


In [ ]:
#With these 2 datasets, when we have the Kaggle data we want to keep that but we want to fill in with TN data for films we don't have in our Kaggle dataset.